# 3D U-Net Model for Predicting Nodules from Lung CT

In [1]:
import numpy as np
import keras
import h5py

from matplotlib import pyplot as plt
%matplotlib inline

/home/bduser/anaconda2/envs/tf18_mkl/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load the trained Keras model.

In [ ]:
filename = "final_unet3d_model.hdf5"
model = keras.models.load_model(filename)

# Load the CT Scans
This is from the hdf5 file of extracted patches. The scan is 256 mm x 256 mm x 128 mm. It takes roughly 35 GB of RAM in order to make a prediction on the entire volume at once.

In [ ]:
ct_scan = h5py.File("data/256x256x128-patch-withdiam.hdf5", "r")

In [ ]:
list(ct_scan.keys())

In [ ]:
class1idx = []
class0idx = []
for idx in range(390):
    if ct_scan["output"][idx] == [1]:
        class1idx.append(idx)
    else:
        class0idx.append(idx)

In [ ]:
class1idx

In [ ]:
def save_imgs(img, prediction_mask, classidx, classtype):
    
    origin = "lower"
    
    for sliceNo in range(max(img.shape)):


        plt.figure(figsize=(15,15))


        plt.subplot(2,3,1)
        plt.imshow(img[sliceNo,:,:,0], cmap="bone", origin=origin)
        plt.title("Original Image", fontsize=18)
        plt.text(-40,32,"Transverse\n    plane", fontsize=32, color="blue", fontname="FreeSans")

        plt.subplot(2,3,2)
        plt.imshow(prediction_mask[0,sliceNo,:,:,0], cmap="bone", origin=origin)
        plt.title("Tumor Prediction", fontsize=18, fontname="FreeSans")

        plt.subplot(2,3,3)
        plt.imshow(img[sliceNo,:,:,0], cmap="bone", origin=origin)
        plt.imshow(prediction_mask[0,sliceNo,:,:,0], alpha=0.5, cmap="Reds", origin=origin)
        #plt.colorbar()
        plt.title("Overlay", fontsize=18, fontname="FreeSans")

        plt.subplot(2,3,4)
        plt.imshow(img[:,sliceNo,:,0], cmap="bone", origin=origin)
        plt.title("Original Image", fontsize=18)
        plt.text(-37,32,"Sagittal\n  plane", fontsize=32, color="blue", fontname="FreeSans")
        plt.xlabel("mm", fontsize=18, fontname="FreeSans")

        plt.subplot(2,3,5)
        plt.imshow(prediction_mask[0,:,sliceNo,:,0], cmap="bone", origin=origin)
        plt.title("Tumor Prediction", fontsize=18, fontname="FreeSans")
        plt.xlabel("mm", fontsize=18, fontname="FreeSans")

        plt.subplot(2,3,6)
        plt.imshow(img[:,sliceNo,:,0], cmap="bone", interpolation="none", origin=origin)
        plt.xlabel("mm", fontsize=18, fontname="FreeSans")

        plt.imshow(prediction_mask[0,:,sliceNo,:,0], alpha=0.5, cmap="Reds", origin=origin)
        #plt.colorbar()
        plt.title("Overlay", fontsize=18, fontname="FreeSans")

        plt.tight_layout()

        import os
        directory = "saved_imgs/class{}/candidate{}/".format(classtype,classidx)
        if not os.path.exists(directory):
            os.makedirs(directory)
        plt.savefig("{}slice{}".format(directory, sliceNo), dpi=600, bbox_inches="tight")

        plt.close()


## It takes roughly 35 GB and 20 seconds to make a prediction on this volume.

The predictions have been saved to the file fullscan_prediction.npz. If you don't have enough memory to make the prediction then skip the next cell and do the np.load instead.

In [ ]:
%%time

img = ct_scan["input"][130,:].reshape(128,256,256,1)  

# centerX = 64
# centerY = 64
# centerZ = 64
# extentX = 96//2
# extentY = 96//2
# extentZ = 96//2
centerX = 64
centerY = 128
centerZ = 128
extentX = 128//2
extentY = 256//2
extentZ = 256//2
img = img[(centerX-extentX):(centerX+extentX),(centerY-extentY):(centerY+extentY),(centerZ-extentZ):(centerZ+extentZ)]

predict_array = model.predict(np.expand_dims(img,0), verbose=2)

prediction_mask = np.round(predict_array[0])
feature_maps = np.round(predict_array[1])

In [ ]:

#feature_maps, prediction_mask, imgs = np.load("fullscan_predict.npz")
#np.savez("fullscan_predict.npz", img=img, prediction_mask=prediction_mask, feature_maps=feature_maps)

In [ ]:
def plotpatch(sliceNo):
    
    m=4 #3,4
    origin = "bottom"

    plt.figure(figsize=(15,15))

    plt.subplot(3,3,1)
    plt.imshow(img[sliceNo,:,:,0], cmap="bone", origin=origin)
    plt.title("Original Image", fontsize=18)
    
    plt.subplot(3,3,2)
    plt.imshow(prediction_mask[0,sliceNo,:,:,0], cmap="bone", origin=origin)
    plt.title("Tumor Prediction", fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,3)
    plt.imshow(img[sliceNo,:,:,0], cmap="bone", origin=origin)
    plt.imshow(feature_maps[0,sliceNo,:,:,m], alpha=0.5, cmap="Reds", origin=origin)
    plt.title("Feature Map #{}".format(m), fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,4)
    plt.imshow(img[:,sliceNo,:,0], cmap="bone", origin=origin)
    plt.title("Original Image", fontsize=18)
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,5)
    plt.imshow(prediction_mask[0,:,sliceNo,:,0], cmap="bone", origin=origin)
    plt.title("Tumor Prediction", fontsize=18, fontname="FreeSans")
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,6)
    plt.imshow(img[:,sliceNo,:,0], cmap="bone", interpolation="none", origin=origin)
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.imshow(feature_maps[0,:,sliceNo,:,m], alpha=0.5, cmap="Reds", origin=origin)
    plt.title("Feature map #{}".format(m), fontsize=18, fontname="FreeSans")
    
    plt.subplot(3,3,7)
    plt.imshow(img[:,:,sliceNo,0], cmap="bone", origin=origin)
    plt.title("Original Image", fontsize=18)
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,8)
    plt.imshow(prediction_mask[0,:,:,sliceNo,0], cmap="bone", origin=origin)
    plt.title("Tumor Prediction", fontsize=18, fontname="FreeSans")
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.subplot(3,3,9)
    plt.imshow(img[:,:,sliceNo,0], cmap="bone", interpolation="none", origin=origin)
    plt.xlabel("mm", fontsize=18, fontname="FreeSans")

    plt.imshow(feature_maps[0,:,:,sliceNo,m], alpha=0.5, cmap="Reds", origin=origin)
    plt.title("Feature map #{}".format(m), fontsize=18, fontname="FreeSans")

    plt.tight_layout()
  

In [ ]:
plotpatch(64)        # 48, 64, 73